### Let's find out what Pakistan talks about and what are our major issues as one can see from news.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams["figure.figsize"] = (16,10)

import string
from dateutil.parser import parse
import calendar

#To perform nlp tasks, use the nltk library.
import nltk
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import gensim
import operator

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/pakistan-news-headlines/27k News Headlines 1 Jan 2020 - Mar 11 2021.csv')

In [ ]:
#This is the dataset's actual form.
df.shape

In [ ]:
df.head(5)

### EDA

In [ ]:
#I've changed the column names and removed the NAN value rows in this cell.
df1 = df.rename(columns = {df.columns[0]:'heading',df.columns[1]:'story',df.columns[2]:'section',df.columns[3]:'timestamp'})
df1.dropna(inplace=True)
df1.reset_index(inplace = True, drop=True)

In [ ]:
def is_date(string):
    try: 
        parse(string, False)
        return True
    except ValueError:
        return False
    
def is_heading(df,string):
    df_heading = df[df['heading'].str.len() < 15]
    unique = df_heading['heading'].unique()
    for uni in unique:
        if string==uni:
            return True
    return False

In [ ]:
def swapping(dfOrignal):
    df = dfOrignal
    
    #Timestamp
    index = df.index
    for idx in index:
        if is_date(df['heading'].iloc[idx]):
            df.loc[idx, ['heading','timestamp']] = df.loc[idx,['timestamp','heading']].values
        elif is_date(df['story'].iloc[idx]):
            df.loc[idx, ['story','timestamp']] = df.loc[idx,['timestamp','story']].values
        elif is_date(df['section'].iloc[idx]):
            df.loc[idx, ['section','timestamp']] = df.loc[idx,['timestamp','section']].values
            
    #Heading
    df_heading = df[df['heading'].str.len() < 15]
    unique = df_heading['heading'].unique()
    index = df.loc[df['section'].isin(unique)].index
    for idx in index:
        df.loc[idx, ['heading','section']] = df.loc[idx,['section','heading']].values
        
    #Story
    index = df.loc[df['story'].isin(unique)].index
    for idx in index:
        if not is_heading(dfOrignal,df['heading'].iloc[idx]):
            df.loc[idx, ['heading','story']] = df.loc[idx,['story','heading']].values
            
    return df

In [ ]:
#I need to swap some of the column values in this cell.
#This algorithm was created because some of the heading column values are in sections and some of the section values are in timestamps. 
df2 = swapping(df1)

In [ ]:
index = df2[df2['timestamp'].str.len()<20].index
df2.drop(index, inplace=True)
df2.reset_index(inplace = True, drop=True)

In [ ]:
df2['date'] = pd.to_datetime(df2['timestamp']).dt.date
df2.drop(['timestamp'],axis=1,inplace=True)

In [ ]:
df2.head()

In [ ]:
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
df2['heading']=df2['heading'].str.replace("'",'')

### Visualization

In [ ]:
#Stop Words
lst_stop_words = ["pakistan","a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "afterwards", "ah", "almost", "alone", "along", "already", "also", "although", "always", "among", "amongst", "announce", "another", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "arent", "arise", "around", "aside", "ask", "asking", "auth", "available", "away", "awfully", "b", "back", "became", "become", "becomes", "becoming", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "believe", "beside", "besides", "beyond", "biol", "brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "couldnt", "date", "different", "done", "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "former", "formerly", "forth", "found", "four", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit", "however", "hundred", "id", "ie", "im", "immediate", "immediately", "importance", "important", "inc", "indeed", "index", "information", "instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug", "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere", "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old", "omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside", "overall", "owing", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "said", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "shed", "shes", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "sufficiently", "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre", "think", "thou", "though", "thoughh", "thousand", "throug", "throughout", "thru", "thus", "til", "tip", "together", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went", "werent", "whatever", "what'll", "whats", "whence", "whenever", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "whim", "whither", "whod", "whoever", "whole", "who'll", "whomever", "whos", "whose", "widely", "willing", "wish", "within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes", "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows", "apart", "appear", "appreciate", "appropriate", "associated", "best", "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning", "consequently", "consider", "considering", "corresponding", "course", "currently", "definitely", "described", "despite", "entirely", "exactly", "example", "going", "greetings", "hello", "help", "hopefully", "ignored", "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar", "it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second", "secondly", "sensible", "serious", "seriously", "sure", "t's", "third", "thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl", "pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol", "el", "los", "pp", "u201d", "well-b", "http", "volumtype", "par", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "ac", "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw", "ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj", "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp", "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc", "dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt", "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej", "el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex", "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy", "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij", "il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr", "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc", "lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms", "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr", "ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol", "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1", "p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm", "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra", "rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si", "sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1", "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm", "tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk", "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo", "vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"]

In [ ]:
#Tokenization Method
def get_tok(df,column,month,year,words_str):
    if month==0 and year==0:
        df_temp = df[df[column].str.contains(words_str, case=False)]
    elif month!="all" and year!="all":
        df_temp = df[(pd.to_datetime(df['date']).dt.month == month) & (pd.to_datetime(df['date']).dt.year == year)]
    else:
        df_temp = df
    sentences = df_temp[column].str.cat(sep = '')
    tok = word_tokenize(sentences)
    tok = [token.lower() for token in tok]
    string.punctuation
    tok = [char for char in tok if char not in string.punctuation]
    stop = gensim.parsing.preprocessing.STOPWORDS.union(set(lst_stop_words))
    tok = [token for token in tok if token not in stop]
    tok = [word for word in tok if len(word) >= 3]
    return tok

In [ ]:
#World Cloud  Method
def world_cloud(df,column,month,year,img,title="",words_str=""):
    mask = np.array(Image.open(img))
    wc = WordCloud(max_font_size=100, max_words=2000, background_color="white",mask=mask)
    wordcloud = wc.generate_from_text(' '.join(get_tok(df,column,month,year,words_str)))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")

    plt.title(title, fontsize=18)
    plt.show()

In [ ]:
#Pakistan
world_cloud(df2,'story',"all","all",'../input/images/pakistan.jpg',"Pakistan News Headlines","")

In [ ]:
#Government
world_cloud(df2,'story',0,0,'../input/images/govt.jpg',"Government Of Pakistan","pakistan|pak|govt|governemnt|country|economy|nation|province")

In [ ]:
#Coronavirus
world_cloud(df2,'story',0,0,'../input/images/covid.png',"Covid-19","coronavirus|pandemic|symptoms|vaccine|covid-19|virus|masks|lockdown|n95|hospital")

In [ ]:
#Issues
world_cloud(df2,'story',0,0,'../input/images/issues.png',"Issues","police|kill|killed|law|issues|issue|court|rape|wheat|sugar|fight|hospital|water")

In [ ]:
#Education
world_cloud(df2,'story',0,0,'../input/images/education.jpg',"Education","education|schools|school|college|university|unversities|classes|exam|papers|board|hec|book|copy|books|copies|student|students|class")

In [ ]:
def show_graph(df,column):
    plt.ion()
    lst = df2['date'].dt.strftime("%Y-%m").unique().tolist()
    lst.sort()
    for dt in lst:
        plt.figure(figsize=(16, 6))
        Freq_dist_nltk=nltk.FreqDist(get_tok(df,column,int(dt[-2:]),int(dt[:4]),""))
        plt.title(str(calendar.month_name[int(dt[-2:])]) + " - "+ str(dt[:4]), fontsize=18)
        Freq_dist_nltk.plot(50, cumulative=False);

In [ ]:
#This graph displays monthly records for the most common terms used in the story column.
show_graph(df2,'story')

In [ ]:
def date_group(df):
    lst_date = df['date'].dt.strftime("%Y-%m").unique().tolist()
    lst_date.sort()
    data = {}
    for dt in lst_date:
        lst = []
        heading = df2['heading'].unique().tolist()
        for h in heading:
            lst.append(len(df[(df['heading']==h) & (df['date']==dt)]))
        data[dt]=lst
    return data

In [ ]:
def show_heading_graph(df):
    data = date_group(df)
    labels = df['heading'].unique().tolist()
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(len(data), 1, figsize=(16, 120))
    count = 0 
    for pt in data.keys():
        ax[count].bar(x - width/2, data.get(pt), label = pt)
        ax[count].set_xticks(x)
        ax[count].set_xticklabels(labels, rotation=50)
        ax[count].legend()
        count+=1
    plt.show()

In [ ]:
#Month-by-month graph of heading column values
show_heading_graph(df2)

In [ ]:
#Based on news headlines, a day-by-day graph has been developed.
df2['day_of_week'] = df2['date'].dt.day_name()
days = df2[['heading','day_of_week']]
ax = days.groupby(days['day_of_week'],sort=False,).count().plot(kind="bar", color = 'g', figsize=(16,6))
plt.show()

<h1 style='color:purple' align='center'>Thank you so much for taking the time to look at my notebook. Please upvote if you like my Notebook.<h1/>